In [17]:
""" Test the water stress calculation for Aqueduct 3.1
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20200205
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:
    TESTING (Boolean) : Toggle testing case.
    SCRIPT_NAME (string) : Script name.
    OUTPUT_VERSION (integer) : output version.
    DATABASE_ENDPOINT (string) : RDS or postGreSQL endpoint.
    DATABASE_NAME (string) : Database name.
    TABLE_NAME_AREA_30SPFAF06 (string) : Table name used for areas. Must exist
        on same database as used in rest of script.
    S3_INPUT_PATH_RIVERDISCHARGE (string) : AWS S3 input path for 
        riverdischarge.    
    S3_INPUT_PATH_DEMAND (string) : AWS S3 input path for 
        demand.     

"""

TESTING = 0
OVERWRITE_OUTPUT = 1
SCRIPT_NAME = 'Y2020M02D05_RH_Aqueduct31_WS_test_V01'
OUTPUT_VERSION = 1

BASIN = 216041 # Normal basin (Ebro)
# BASIN = 742826 # Basin with negative final water stress values in February
# BASIN = 635303 # Basin with negative water stress in february 1962
# BASIN = 291707 # Basin with water stress exceedign 1 in february


DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d09_rh_apply_aridlowonce_mask_postgis_v01_v05"
OUTPUT_TABLE_NAME = SCRIPT_NAME.lower() + "_v{:02.0f}".format(OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME, 
      "\nOutput Table: " , OUTPUT_TABLE_NAME)

Input Table:  y2018m07d09_rh_apply_aridlowonce_mask_postgis_v01_v05 
Output Table:  y2020m02d05_rh_aqueduct31_ws_test_v01_v01


In [18]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2020M02D06 UTC 13:04


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [19]:
# imports
import re
import os
import numpy as np
import pandas as pd
import aqueduct3
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [20]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))
#connection = engine.connect()

if OVERWRITE_OUTPUT:
    sql = "DROP TABLE IF EXISTS {};".format(OUTPUT_TABLE_NAME)
    print(sql)
    result = engine.execute(sql)

DROP TABLE IF EXISTS y2020m02d05_rh_aqueduct31_ws_test_v01_v01;


sql = "SELECT"
sql += " *"
sql += " FROM {}".format(INPUT_TABLE_NAME)
sql += " WHERE pfafid_30spfaf06 = {}".format(BASIN)
sql += " AND temporal_resolution = 'month'"
sql += " AND month = 2"

In [24]:
sql = "SELECT"
sql += " *"
sql += " FROM {}".format(INPUT_TABLE_NAME)
sql += " WHERE pfafid_30spfaf06 = {}".format(BASIN)
sql += " AND year = 2014"

In [25]:
sql

'SELECT * FROM y2018m07d09_rh_apply_aridlowonce_mask_postgis_v01_v05 WHERE pfafid_30spfaf06 = 216041 AND year = 2014'

In [26]:
df_raw = pd.read_sql(sql=sql,con=engine)

In [27]:
df_raw.dtypes

pfafid_30spfaf06                                                   int64
temporal_resolution                                               object
year                                                               int64
month                                                              int64
area_m2_30spfaf06                                                float64
area_count_30spfaf06                                               int64
pdomww_m_30spfaf06                                               float64
pdomwn_m_30spfaf06                                               float64
pindww_m_30spfaf06                                               float64
pindwn_m_30spfaf06                                               float64
pirrww_m_30spfaf06                                               float64
pirrwn_m_30spfaf06                                               float64
plivww_m_30spfaf06                                               float64
plivwn_m_30spfaf06                                 

In [28]:
ptotww = df_raw[["year",
                 "ols10_ptotww_m_30spfaf06",
                 "capped_ols10_ptotww_m_30spfaf06",
                 "ols10_riverdischarge_m_30spfaf06",
                 "capped_ols10_riverdischarge_m_30spfaf06",
                 "ols10_waterstress_dimensionless_30spfaf06",
                 "capped_ols10_waterstress_dimensionless_30spfaf06",
                 "ols_capped_ols10_waterstress_dimensionless_30spfaf06"]]

In [30]:
ptotww

,year,ols10_ptotww_m_30spfaf06,capped_ols10_ptotww_m_30spfaf06,ols10_riverdischarge_m_30spfaf06,capped_ols10_riverdischarge_m_30spfaf06,ols10_waterstress_dimensionless_30spfaf06,capped_ols10_waterstress_dimensionless_30spfaf06,ols_capped_ols10_waterstress_dimensionless_30spfaf06
0,2014,0.003045,0.003045,0.468768,0.468768,0.006496,0.006496,0.007096
1,2014,0.003449,0.003449,0.449186,0.449186,0.007677,0.007677,0.009204
2,2014,0.003723,0.003723,0.654907,0.654907,0.005685,0.005685,0.009475
3,2014,0.003417,0.003417,0.629603,0.629603,0.005427,0.005427,0.009924
4,2014,0.004566,0.004566,0.558619,0.558619,0.008174,0.008174,0.010300
5,2014,0.007000,0.007000,0.529602,0.529602,0.013218,0.013218,0.026947
6,2014,0.011914,0.011914,0.329903,0.329903,0.036113,0.036113,0.066014
7,2014,0.014243,0.014243,0.170613,0.170613,0.083480,0.083480,0.132925
8,2014,0.008494,0.008494,0.073821,0.073821,0.115064,0.115064,0.107159
9,2014,0.006166,0.006166,0.157167,0.157167,0.039233,0.039233,0.029474
